In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import numpy as np

In [27]:
df = pd.read_csv("./dataset/movies.csv")
genres = pd.read_csv("./dataset/genres.csv")

In [28]:
df.head()

,id,name,date,tagline,description,minute,rating
0,1000001,Barbie,2023.0,She's everything. He's just Ken.,Barbie and Ken are having the time of their li...,114.0,3.91
1,1000002,Parasite,2019.0,Act like you own the place.,"All unemployed, Ki-taek's family takes peculia...",133.0,4.57
2,1000003,Everything Everywhere All at Once,2022.0,The universe is so much bigger than you realize.,An aging Chinese immigrant is swept up in an i...,140.0,4.32
3,1000004,Fight Club,1999.0,Mischief. Mayhem. Soap.,A ticking-time-bomb insomniac and a slippery s...,139.0,4.27
4,1000005,Interstellar,2014.0,Mankind was born on Earth. It was never meant ...,The adventures of a group of explorers who mak...,169.0,4.32


In [29]:
genres.head()

,id,genre
0,1000001,Comedy
1,1000001,Fantasy
2,1000001,Adventure
3,1000002,Comedy
4,1000002,Thriller


In [30]:
movies = df.drop(columns=["date", "tagline", "minute", "minute"])

In [31]:
df_all = movies.merge(genres.drop_duplicates(), on=['id'], 
                   how='left', indicator=True)

In [36]:
df_all.loc[df_all['_merge'] == 'both']

,id,name,description,rating,genre,_merge
0,1000001,Barbie,Barbie and Ken are having the time of their li...,3.91,Comedy,both
1,1000001,Barbie,Barbie and Ken are having the time of their li...,3.91,Fantasy,both
2,1000001,Barbie,Barbie and Ken are having the time of their li...,3.91,Adventure,both
3,1000002,Parasite,"All unemployed, Ki-taek's family takes peculia...",4.57,Comedy,both
4,1000002,Parasite,"All unemployed, Ki-taek's family takes peculia...",4.57,Thriller,both
...,...,...,...,...,...,...
1248536,1896380,Никогда не сдавайся,NaN,NaN,Crime,both
1248537,1896380,Никогда не сдавайся,NaN,NaN,Drama,both
1248539,1896382,Ավլոս,NaN,NaN,Drama,both
1248548,1896391,柴咲コウ CONCERT TOUR 2023 ACTOR'S THE BEST,KO SHIBASAKI CONCERT TOUR 2023 - ACTOR'S THE BEST,NaN,Music,both
